In [2]:
# Cuts used to go from summary.fits to cut.fits, for now we will use the cut.fits file

#snr = 5.0
#sharp = 0.04
#crowd = 0.5
#objtype = 1
#flag = 99
#importing modules

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.coordinates import match_coordinates_sky
import numpy as np
import os
import glob 

In [12]:
#DECLARING DIRECTORIES AND SEARCH PROPERTIES
#path to folder of summary/cut.fits with format sn_name-filter-mjd_#-cut/sum.fits i.e: 2021gmj-F625W-mjd_59574-cut.fits
path_sum = 'C:\\Users\\domdo\\Lamat\\Foley\\photometry\\summary\\'
#path to folder of summary/cut.fits with format sn_name-filter-mjd_#-cut/sum.fits i.e: 2021gmj-F625W-mjd_59574-cut.fits
path_cut = 'C:\\Users\\domdo\\Lamat\\Foley\\photometry\\cut\\'
#path to base folder to save table
table_path = 'C:\\Users\\domdo\\Lamat\\Foley\\photometry\\table\\'
#where full region files will be written
outpath_full = 'C:\\Users\\domdo\\Lamat\\Foley\\photometry\\final_catalogs_full\\'
#where cut region files will be written
outpath_cut = 'C:\\Users\\domdo\\Lamat\\Foley\\photometry\\final_catalogs_cut\\'
#position in ra and dec (degrees)
ra = 309.33000691667
dec = 66.106417016667
#search radius (arc seconds)
radius = 3

In [15]:
#writes region files to outpath
#the centre of ds9 pixel is (1,1), but centre of dolphot pixel is (0.5,0.5)
#to make it in ds9 format it’s (+0.5,+0.5) to dolphot coordinates (? i dont think this is true anymore, removing it matches the region file coords with the -cut.fits table)
def writeregionfile(filename, outpath, objlist, radius, color="blue", sys=''):
    if sys == '': 
        sys = 'wcs'
    completeName = os.path.join(outpath, filename)
    out = open(completeName, 'w')
    i = -1
    out.write('# Region file format: DS9 version 4.0\nglobal color='+ color +' font="helvetica 10 normal" select=1 highlite=1 edit=1 move=1 delete=1 include=1 fixed=0 source\n')
    if sys == 'wcs':
        out.write('fk5\n')
        for i in range(len(objlist)):
            out.write("point(%.7f,%.7f) # point=boxcircle text={%i}\n" % (objlist.RA[i],objlist.DEC[i], i))
        if objlist.RA.size == 0:
            print("ERROR: No stars detected. Increase search radius or double check position.")
            sys.exit()
        out.write("circle(" + str(objlist.RA[0]) + "," + str(objlist.DEC[0]) + "," + str(radius/3600) +")\n")#1 degree is 3600 arcsecs
    if sys == 'img':
        out.write('image\n')
        for i in range(len(objlist)):
            out.write("point(%.3f,%.3f) # point=boxcircle text={%i}\n" % (objlist.X[i],objlist.Y[i], i))
        if objlist.RA.size == 0:
            print("ERROR: No stars detected. Increase search radius or double check position.")
            sys.exit()
        out.write("circle(" + str(objlist.X[0]) + "," + str(objlist.Y[0]) + "," + str(radius * 25.239777) +")\n")#25.239777 is 1 arcsec in physical
    out.close()
    print("writing region file:", filename)


In [16]:
#finds sn name, filter, mjd (date), and search radius
def find_attributes(filename, radius):
    global sn_name
    global filt
    global mjd
    global search_radius
    #reset attributes
    sn_name = ''
    filt = ''
    mjd = ''
    search_radius = 0
    #split name
    filename_split = filename.split("-")#split filename into attributes
    
    #find name
    sn_name = filename_split[0]
    print("SN name: " + sn_name)
    #find filt
    filt = filename_split[1]
    print("filt: " + filt)
    #find mjd    
    mjd = filename_split[2]
    print("MJD: " + mjd)
    
    search_radius = radius * u.arcsec
    print('Search radius: ' + str(search_radius))

In [17]:
#creates cut region files using cut.fits and table of all region points properties
#rai, deci is ra and dec of position, radius is search radius
def returnregionfilescut(path, rai, deci, radius, outpath, table_path): #use this for cut.fits
    dir_list = glob.glob(path)
    filenames = [os.path.basename(x) for x in glob.glob(path+"*-cut.fits")]
    count = 0
    #for each file in path
    for count, filename in enumerate(filenames):
        if filename != '.DS_Store': #weird mac meta file
            find_attributes(filename, radius)
            sn_position = SkyCoord(rai*u.degree, +deci*u.degree)
            cut_catalog=fits.open(path+filename, ignore_missing_simple=True)[1]
            data_cut=cut_catalog.data
            tbl_cut=Table(data_cut)
            radec_cut = SkyCoord(ra=data_cut.RA*u.degree, dec=data_cut.DEC*u.degree)
            d2d_cut = sn_position.separation(radec_cut)
            index_cut= d2d_cut < search_radius
            data_searched_cut=data_cut[index_cut]
            tbl_searched_cut=tbl_cut[index_cut]
            writeregionfile(sn_name+"-"+filt+"-"+mjd+"-"+str(search_radius.value) +"_arcsec-ra_dec_cut.reg", outpath, data_searched_cut, radius, "red", "wcs")
            writeregionfile(sn_name+"-"+filt+"-"+mjd+"-"+str(search_radius.value) +"_arcsec-pixel_xy_cut.reg", outpath, data_searched_cut, radius, "red", "img")
            tbl_searched_cut.add_column(np.arange(len(tbl_searched_cut)), name= "Star #", index  = 0)
            tbl_searched_cut.add_column(tbl_searched_cut['X'] + 0.5, name = 'DS9_X', index = 6)
            tbl_searched_cut.add_column(tbl_searched_cut['Y'] + 0.5, name = 'DS9_Y', index = 7)
            tbl_searched_cut['DS9_X'] = np.around(tbl_searched_cut['DS9_X'], decimals = 3)
            tbl_searched_cut['DS9_Y'] = np.around(tbl_searched_cut['DS9_Y'], decimals = 3)
            tbl_searched_cut.write(table_path+sn_name+"-"+filt+"-"+mjd+"-"+str(search_radius.value)+'_arcsec_cut_tbl'+'.html', format = 'html', overwrite = True)

In [18]:
#creates full region files using summary.fits
#rai, deci is ra and dec of position, radius is search radius
def returnregionfilesfull(path, rai, deci, radius, outpath, table_path): #use this for summary.fits
    dir_list = glob.glob(path)
    filenames = [os.path.basename(x) for x in glob.glob(path+"*-summary.fits")]
    count = 0
    #for each file in path
    for count, filename in enumerate(filenames):
        if filename != '.DS_Store': #weird mac meta file
            print(filename)
            find_attributes(filename, radius) #get name, mjd, filt
            sn_position = SkyCoord(rai*u.degree, +deci*u.degree)
            full_catalog=fits.open(path+filename, ignore_missing_simple=True)[1]
            data_full=full_catalog.data
            tbl_full=Table(data_full)
            radec_full = SkyCoord(ra=data_full.RA*u.degree, dec=data_full.DEC*u.degree)
            d2d_full = sn_position.separation(radec_full)
            index_full= d2d_full < search_radius
            data_searched_full=data_full[index_full]
            tbl_searched_full=tbl_full[index_full]
            writeregionfile(sn_name+"-"+filt+"-"+mjd+"-"+str(search_radius.value) +"_arcsec-ra_dec_full.reg", outpath, data_searched_full, radius, "red", "wcs")
            writeregionfile(sn_name+"-"+filt+"-"+mjd+"-"+str(search_radius.value) +"_arcsec-pixel_xy_full.reg", outpath, data_searched_full, radius, "red", "img")
            tbl_searched_full.add_column(np.arange(len(tbl_searched_full)), name= "Star #", index  = 0)
            tbl_searched_full.add_column(tbl_searched_full['X'] + 0.5, name = 'DS9_X', index = 6)
            tbl_searched_full.add_column(tbl_searched_full['Y'] + 0.5, name = 'DS9_Y', index = 7)
            tbl_searched_full['DS9_X'] = np.around(tbl_searched_full['DS9_X'], decimals = 3)
            tbl_searched_full['DS9_Y'] = np.around(tbl_searched_full['DS9_Y'], decimals = 3)
            tbl_searched_full.write(table_path+sn_name+"-"+filt+"-"+mjd+"-"+str(search_radius.value)+'_arcsec_full_tbl'+'.html', format = 'html', overwrite = True)

In [19]:
#returnregionfiles(path, ra, dec, search radius)
#returning region files of summary.fits
#path should point to folder of summary.fits in format previously stated
#SN2021sjt_summary
returnregionfilesfull(path_sum, ra, dec, radius, outpath_full, table_path)

2021sjt-F110W-mjd_57452-summary.fits
SN name: 2021sjt
filt: F110W
MJD: mjd_57452
Search radius: 3.0 arcsec
writing region file: 2021sjt-F110W-mjd_57452-3.0_arcsec-ra_dec_full.reg
writing region file: 2021sjt-F110W-mjd_57452-3.0_arcsec-pixel_xy_full.reg
2021sjt-F128N-mjd_57452-summary.fits
SN name: 2021sjt
filt: F128N
MJD: mjd_57452
Search radius: 3.0 arcsec
writing region file: 2021sjt-F128N-mjd_57452-3.0_arcsec-ra_dec_full.reg
writing region file: 2021sjt-F128N-mjd_57452-3.0_arcsec-pixel_xy_full.reg
2021sjt-F218W-mjd_49549-summary.fits
SN name: 2021sjt
filt: F218W
MJD: mjd_49549
Search radius: 3.0 arcsec
writing region file: 2021sjt-F218W-mjd_49549-3.0_arcsec-ra_dec_full.reg
writing region file: 2021sjt-F218W-mjd_49549-3.0_arcsec-pixel_xy_full.reg
2021sjt-F547M-mjd_49550-summary.fits
SN name: 2021sjt
filt: F547M
MJD: mjd_49550
Search radius: 3.0 arcsec
writing region file: 2021sjt-F547M-mjd_49550-3.0_arcsec-ra_dec_full.reg
writing region file: 2021sjt-F547M-mjd_49550-3.0_arcsec-pixel_

In [20]:
#returnregionfiles(path, ra, dec, search radius)
#returning region files of cut.fits
#path should point to folder of cut.fits in format previously stated
#SN2021sjt_cut
returnregionfilescut(path_cut, ra, dec, radius, outpath_cut, table_path)

SN name: 2021sjt
filt: F110W
MJD: mjd_57452
Search radius: 3.0 arcsec
writing region file: 2021sjt-F110W-mjd_57452-3.0_arcsec-ra_dec_cut.reg
writing region file: 2021sjt-F110W-mjd_57452-3.0_arcsec-pixel_xy_cut.reg
SN name: 2021sjt
filt: F128N
MJD: mjd_57452
Search radius: 3.0 arcsec
writing region file: 2021sjt-F128N-mjd_57452-3.0_arcsec-ra_dec_cut.reg
writing region file: 2021sjt-F128N-mjd_57452-3.0_arcsec-pixel_xy_cut.reg
SN name: 2021sjt
filt: F218W
MJD: mjd_49549
Search radius: 3.0 arcsec
writing region file: 2021sjt-F218W-mjd_49549-3.0_arcsec-ra_dec_cut.reg
writing region file: 2021sjt-F218W-mjd_49549-3.0_arcsec-pixel_xy_cut.reg
SN name: 2021sjt
filt: F547M
MJD: mjd_49550
Search radius: 3.0 arcsec
writing region file: 2021sjt-F547M-mjd_49550-3.0_arcsec-ra_dec_cut.reg
writing region file: 2021sjt-F547M-mjd_49550-3.0_arcsec-pixel_xy_cut.reg
SN name: 2021sjt
filt: F555W
MJD: mjd_52037
Search radius: 3.0 arcsec
writing region file: 2021sjt-F555W-mjd_52037-3.0_arcsec-ra_dec_cut.reg
wri